Here we are going to look at **git as purely local file management**.  The next one looks at working with the server. 

**Before we start** remove any pre-existing version of the dummy project that we'll create and see what our git configuration looks like.    
_Note_ Sometimes running in a notebook git can take up to a minute to return. We also use `Out-string` for nicer formating in a notebook. This isn't necessary at normal prompt

In [ ]:
cd ~
del dummyProj -rec -force -ErrorAction SilentlyContinue
git config --list | out-string

http.sslbackend=schannel
diff.astextplain.textconv=astextplain
core.autocrlf=true
core.fscache=true
core.symlinks=false
core.editor="C:\\Program Files\\Microsoft VS Code\\Code.exe" --wait
credential.helper=manager-core
filter.lfs.clean=git-lfs clean -- %f
filter.lfs.smudge=git-lfs smudge -- %f
filter.lfs.process=git-lfs filter-process
filter.lfs.required=true
pull.rebase=false
credential.https://dev.azure.com.usehttppath=true
init.defaultbranch=master
user.name=James O'Neill
user.email=P10506111@capita.co.uk
credential.helper=wincred


Lets **Initialize a new local repo**.    
_What is a "repo"_ - a **Repository** is a collection of files being tracked by Git - the files and folders under a given path; the path might be a directory on a local disk, or a URL to a server. 
Within the repo git knows about "blobs" which are versions of files (current or historic) and knows the blob that represents the _active_ version of each file in the repo. As well as knowing which blobs are present as which files _now_ Git knows what was present (and where) at differnt points in time (giving us the ability to move backwards and forwards in file history) 

So let's create a one:
- make a directory, 
- move into it, 
- run `git init`

In [ ]:
cd ~
$null = md dummyProj 
cd ~\dummyProj  
git init

Initialized empty Git repository in C:/Users/P10506111/dummyProj/.git/

When we add a file to the directory, it is present in the file system, but Git doesn't automatically make the blob for that version. So let's
- make a file,
- use `git add filename` to tell Git to start tracking it - copy the current version into a blob and note that blob corresponds to the file. 
- make a **new commit** with `git commit`. We will add a **commit message**  with `-m` and use `-v` to do the commit verbosly

_What's a **commit** ?_   
When I said before that Git knows which files existed in the repo at differnt points in time and which blobs represented them, a commit is saying "This is one of those points". Commiting is recording the layout of files and directories, and which blob maps to which file at that point in time.  

In [ ]:
cd ~\dummyProj 
echo "Placeholder" > license.md 
git add license.md  
git commit -v -m "Inital version"  | Out-String 

[master (root-commit) dee40e5] Inital version
 1 file changed, 1 insertion(+)
 create mode 100644 license.md


Git keeps a log so let's use `git log`edit see what's in the **log** so far. 

In [ ]:
cd ~\dummyProj 
git log | Out-String

commit dee40e50c555a2ccff7be953f9d2b05a68335dda
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Fri Jan 8 16:34:59 2021 +0000

    Inital version


We can get a shorter view with `--oneline`

In [ ]:
cd ~\dummyProj 
git log --oneline

dee40e5 Inital version

_Side note from https://git-scm.com/book/en/v2/Git-Basics-Getting-a-Git-Repository_

To **clone** the Git linkable library called libgit2, you can do so like this:    
`git clone https://github.com/libgit2/libgit2`    
That creates a directory named `libgit2`, initializes a `.git` directory inside it, pulls down all the data for that repository, and checks out a working copy of the latest version.   
To clone the repository into a directory named something other than libgit2, you can specify the new directory name as an additional argument:    
`git clone https://github.com/libgit2/libgit2 mylibgit`    

Whether we used **clone** or **init** we now have a git enabled folder - in other words one with a .git directory which is where the blobs and commit information live . 

We can check the  **status** of the repo with `git status`

In [ ]:
cd ~/dummyProj 
git status  |Out-String

On branch master
nothing to commit, working tree clean


We will explain what `On branch <name>` means shortly. The Name appeared in settings under `init.defaultbranch` 

The status will change if we add, update or delete files, so let's 
- modify an _existing_ file (by extending it) 
- add a new file (make a copy of an exesting one),
- get the status again.

In [ ]:
cd ~/dummyProj 
echo 'more' >> license.md 
copy license.md readme
git status  | Out-String

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   license.md

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	readme

no changes added to commit (use "git add" and/or "git commit -a")


Git tells us our changes are in the form of 
- one additional  file, currently **untracked**, and 
- one **modified** file which has not yet been **staged** for committing.

`git add` will update what git considers ready-to-commit, and `git restore` will roll back to the last committed version.
`git commit -a` is like doing `git add *` followed by `git commit`

We can see the changes to existing files with `diff`

In [ ]:
git diff | Out-String

diff --git a/license.md b/license.md
index 3b94f91..ce2ca9f 100644
--- a/license.md
+++ b/license.md
@@ -1 +1,2 @@
 Placeholder
+more


`git add *` will **stage** all new and changed files, and we can check the status again.

In [ ]:
git add *  
git status | Out-String

On branch master
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   license.md
	new file:   readme



As before, `restore` can be used to to reverse things, `restore --staged` moves the file back (undoing any changes since `git add` was run) and removes it from the list of files to include in the next commit.

The `diff` command can show us what has changed - on it's own it shows what is *tracked*, *unstaged* files, and all our files are *staged* now so that won't show anything `--cached` shows what's staged and waiting to be committed. 

In [ ]:
git diff --cached | Out-String

diff --git a/license.md b/license.md
index 3b94f91..ce2ca9f 100644
--- a/license.md
+++ b/license.md
@@ -1 +1,2 @@
 Placeholder
+more
diff --git a/readme b/readme
new file mode 100644
index 0000000..ce2ca9f
--- /dev/null
+++ b/readme
@@ -0,0 +1,2 @@
+Placeholder
+more


Let's `commit` the the change  with a new message (`-m`)   
Git commit messages are usually kept quite short, and often contain tracking information to link to pieces of work being done. Otherwise good commit messages tend to be ones which complete the sentance "Incoporate this commit to ___" 

In [ ]:
cd ~/dummyProj
git commit -m "two"  |out-string

[master 9463cdd] two
 2 files changed, 3 insertions(+)
 create mode 100644 readme


 And we see what is in the log after the commit

In [ ]:
git log --oneline | out-string

9463cdd two
dee40e5 Inital version


So the `git log`  **shows the history of every commit** which got us to this point. 

Let's remove a file, and to see the effects of multiple changes we'll also add to an existing file. 

In [ ]:
del .\license.md
echo 'extra' >> .\readme
git status | Out-String

On branch master
Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    license.md
	modified:   readme

no changes added to commit (use "git add" and/or "git commit -a")


This time, the status shows a deleted file. We might be temped to use the `git rm *` command to remove all deleted files, but beware...

In [ ]:
git rm *

error: the following file has local modifications:
    readme
(use --cached to keep the file, or -f to force removal)

`git rm *` tried to remove _everything_ `git add *` adds all files and re-adding existing ones has no effect. `git rm *` tries to remove all the files in the repo!    
Fortunately our modified file saved us. Let's try just remove the file which needs deleting, and add the other changed files.     
Then let's try a new option for the `git status` command. 

In [ ]:
git rm ./license.md
git add *

rm 'license.md'

In [ ]:
git status -s | Out-String

D  license.md
M  readme


to keep the file in your working tree but remove it from your staging area  e.g accidentally staged    
`git rm --cached README`   
https://git-scm.com/book/en/v2/Git-Basics-Recording-Changes-to-the-Repository 

Missed a file ? add it & run `git commit --amend`  

For now lets use `git commit` to commit the deletion(s) and update(s) with `-m` to specify a commit message. Then we can see what has changed in the log

In [ ]:
git commit -m "three"  | out-string

[master 9cc95b3] three
 2 files changed, 1 insertion(+), 2 deletions(-)
 delete mode 100644 license.md


In [ ]:
git log | Out-String 

commit 9cc95b32a558058033af3880ad0aaa20cb283e1e
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Fri Jan 8 16:38:13 2021 +0000

    three

commit 9463cdd7cf2f2fa2c7b9120f879a70d4d1d02ec4
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Fri Jan 8 16:37:13 2021 +0000

    two

commit dee40e50c555a2ccff7be953f9d2b05a68335dda
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Fri Jan 8 16:34:59 2021 +0000

    Inital version


We can move or rename files and follow the operation with    
`git rm <oldName>`    
`git add <NewName>` 
but `git mv` does all 3 in one 

In [ ]:
git mv .\readme read_me.txt  

In [ ]:
git status -s

R  readme -> read_me.txt

git log -p -1

In [ ]:
git log --stat --since=1.hours  | Out-String

commit 9cc95b32a558058033af3880ad0aaa20cb283e1e
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Fri Jan 8 16:38:13 2021 +0000

    three

 license.md | 2 --
 readme     | 1 +
 2 files changed, 1 insertion(+), 2 deletions(-)

commit 9463cdd7cf2f2fa2c7b9120f879a70d4d1d02ec4
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Fri Jan 8 16:37:13 2021 +0000

    two

 license.md | 1 +
 readme     | 2 ++
 2 files changed, 3 insertions(+)

commit dee40e50c555a2ccff7be953f9d2b05a68335dda
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Fri Jan 8 16:34:59 2021 +0000

    Inital version

 license.md | 1 +
 1 file changed, 1 insertion(+)


# Branches
The line of commits forms a _branch_. In the early years of git, the first branch was always named "master". Other names are now allowed and people increasingly prefer other names like "main" or "trunk.    
![Changing the git default branch name](Git_Windows_default_Branch.png)    
We are currently working in the default branch so  the **HEAD** pointer and current state of that branch map to the our most recent commit.  

In [ ]:
cd ~/dummyProj
git log --oneline --decorate 

9cc95b3 (HEAD -> master) three
9463cdd two
dee40e5 Inital version

In this case**HEAD** points to the **master** branch, which is currently at point to the commit with the hash `9cc95b3 ` which had the commit message **three** , this is descended from commit `9463cdd  ` ("two"), which descended from commit `dee40e5  ` ("Initial version") 

If we add a new branch it creates a new name and with no other parameters starts pointing wherever **HEAD** points but **HEAD** does not switch to the new branch. 

In [ ]:
git branch testing

In [ ]:
git log --oneline --decorate  | out-string

9cc95b3 (HEAD -> master, testing) three
9463cdd two
dee40e5 Inital version


Now status shows that HEAD still points to master, and master and the new branch both point to the commit named "three" 

`git checkout <branchname>` selects a branch to work in -     
or we can create & select in one command with  `git checkout -b <branchname>`

In [ ]:
git checkout testing   

A	read_me.txt
Switched to branch 'testing'
D	readme

<u>Now</u> the **head** points to the new branch, and it and master still point to the commit named "three"

In [ ]:
git log --oneline --decorate  | Out-String

9cc95b3 (HEAD -> testing, master) three
9463cdd two
dee40e5 Inital version


If we add a file and commit our changes, the HEAD pointer and the current branch ("testing") move forward to the new commit. But any other branches (like "master") remain at their existing commits.

In [ ]:
cd ~/dummyProj
dir -force > test.txt
git commit -a -m "changed the testing branch" | Out-String

[testing d7a2849] changed the testing branch
 1 file changed, 0 insertions(+), 0 deletions(-)
 rename readme => read_me.txt (100%)


In [ ]:
git log --oneline --decorate | Out-String

d7a2849 (HEAD -> testing) changed the testing branch
9cc95b3 (master) three
9463cdd two
dee40e5 Inital version


We can switch back to the "master" branch, (changing where HEAD points), change its content and commit

In [ ]:
git checkout master

Switched to branch 'master'

In [ ]:
cd ~/dummyProj
echo "new" > update.txt
git add *

In [ ]:
git commit -m "changed master" | Out-String

[master b8c9ef1] changed master
 2 files changed, 11 insertions(+)
 create mode 100644 test.txt
 create mode 100644 update.txt


Another version of the `git log` command shows us both branches and how the overall stucture:

In [ ]:
git log --oneline --decorate --graph --all | Out-String

* b8c9ef1 (HEAD -> master) changed master
| * d7a2849 (testing) changed the testing branch
|/  
* 9cc95b3 three
* 9463cdd two
* dee40e5 Inital version


**HEAD** points to "master" and is currently at commit `b8c9ef1  `; we have another branch "testing" which is at commit `d7a2849  ` and both `b8c9ef1` and `d7a2849  ` are dervived from `9cc95b3 ` which in turn is derived from `9463cdd` and so on

Let's create a branch for a hot fix. 

In [ ]:
git checkout -b hotfix   
echo "hotfix" > hotfix.txt
git add *

Switched to a new branch 'hotfix'

In [ ]:
git commit -m "added my hotfix" > ~\temp.txt

In [ ]:
type ~\temp.txt

[hotfix 9427c6c] added my hotfix
 1 file changed, 1 insertion(+)
 create mode 100644 hotfix.txt

In [ ]:
git log --oneline --decorate --graph --all | out-string 

* 9427c6c (HEAD -> hotfix) added my hotfix
* b8c9ef1 (master) changed master
| * d7a2849 (testing) changed the testing branch
|/  
* 9cc95b3 three
* 9463cdd two
* dee40e5 Inital version


We can build and deploy the hotfix using our new branch as the source of the code.     
Eventually we will want to merge those changes into the "master" branch.
Because the  commit  for "hotfix" is a direct descentant of the commit for "master", this is just a case of moving the name "master" along the line of descent - something git calls "fast forwarding"

In [ ]:
git checkout master   

Switched to branch 'master'

In [ ]:
git   merge hotfix  

Updating b8c9ef1..9427c6c
Fast-forward
 hotfix.txt | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 hotfix.txt

Now **HEAD** is attached to "master" which has moved forward from `b8c9ef1` to `9427c6c`; "hotfix" is still on `9427c6c` as well but it is a redundant label so we can delete it

In [ ]:
git branch -d hotfix 

Deleted branch hotfix (was 9427c6c).

If we look at "testing" and "master", we can see they have a common ancestor `9cc95b3 `. Testing has one extra commit which master doesn't have and lacks two commits which have been made to master - it is said to be "one ahead and two behind" . We can't do a fast forward merge but we can merge the changes from both to make a single new commit with all the changes. 

In [ ]:
git checkout testing 
git merge master  | out-String

Switched to branch 'testing'
Merge made by the 'recursive' strategy.
 hotfix.txt |  1 +
 test.txt   | 10 ++++++++++
 update.txt |  1 +
 3 files changed, 12 insertions(+)
 create mode 100644 hotfix.txt
 create mode 100644 test.txt
 create mode 100644 update.txt

In this case the two branches have made changes which are independent of each other. Let's create a conflict. First let's update a file in the current branch ("Testing")

In [ ]:
type .\update.txt
echo "Added from the testing branch" >> update.txt
git commit -a -m 'Update from testing' | out-string

new
[testing db158c3] Update from testing
 1 file changed, 1 insertion(+)

In [ ]:
git checkout master
type .\update.txt
''
echo "Added from the master branch" >> update.txt
git commit -a -m 'Update from master' | out-string
''
git log --oneline --decorate --graph --all | out-String

Switched to branch 'master'
new

[master 5723e06] Update from master
 1 file changed, 1 insertion(+)


* 5723e06 (HEAD -> master) Update from master
| * db158c3 (testing) Update from testing
| *   417bb08 Merge branch 'master' into testing
| |\  
| |/  
|/|   
* | 9427c6c added my hotfix
* | b8c9ef1 changed master
| * d7a2849 changed the testing branch
|/  
* 9cc95b3 three
* 9463cdd two
* dee40e5 Inital version


As before the two branches have a common ancestor so let's try to merge them 

In [ ]:
git merge testing

Auto-merging update.txt
CONFLICT (content): Merge conflict in update.txt
Automatic merge failed; fix conflicts and then commit the result.

Oh dear - we need to manually edit update.txt which looks like this

In [ ]:
Type update.txt

new
<<<<<<< HEAD
Added from the master branch
Added from the testing branch
>>>>>>> testing

We can edit the file and either run `git merge --continue` (which will ask for the commit message to be edited) or use `git commit -a -m "merge message"`  and the log shows a new commit which brings together the shared ancestor and the updates.

In [ ]:
echo "Combined file" >> updated.txt
git commit -a -m "Merge branch 'testing'"

[master 783454f] Merge branch 'testing'

In [ ]:
git log --oneline --decorate --graph --all | out-string

*   783454f (HEAD -> master) Merge branch 'testing'
|\  
| * db158c3 (testing) Update from testing
| *   417bb08 Merge branch 'master' into testing
| |\  
| * | d7a2849 changed the testing branch
* | | 5723e06 Update from master
| |/  
|/|   
* | 9427c6c added my hotfix
* | b8c9ef1 changed master
|/  
* 9cc95b3 three
* 9463cdd two
* dee40e5 Inital version
